## Task

As an input we have query of a view for gathering statistics.
<br>
The task is to get all of the formulas in the human readable format.
<br><br>
<b>For example:</b>
<br>
'sum(nvl(COUNTER1,0)) + sum(nvl(COUNTER2,0))'   ->   '(COUNTER1+COUNTER2)'
<br><br>
<b>Additional condition:</b><br>
If formulas contain division by counter, there is a mistake in the initial query. Counters like this one should not be added to the final output.

## Initial data

In [1]:
# connecting to the db
from helpers.utils import connect
con, cur = connect('jump')

In [2]:
import pandas as pd
import numpy as np
import moz_sql_parser
import json

from helpers.tools import clean_formula, simplify

In [3]:
with open('clean_sql_hua_2g.txt', 'r') as f:
    q = f.read()

So here is the beginning of the query.

In [4]:
q.split('\n')[:11]

['SELECT ',
 'ROUND((23*SUM(NVL(L9002,0))+34*SUM(NVL(L9003,0))+40*SUM(NVL(L9004,0))+54*SUM(NVL(L9005,0)))/1024,2) TOT_VOLUM_CELL_GPRS_UL_KB,',
 'ROUND((22*SUM(NVL(L9202,0))+28*SUM(NVL(L9203,0))+37*SUM(NVL(L9204,0))+44*SUM(NVL(L9205,0))+56*SUM(NVL(L9206,0))+74*SUM(NVL(L9207,0))+56*SUM(NVL(L9208,0))+68*SUM(NVL(L9209,0))+74*SUM(NVL(L9210,0)))/1024,2) TOT_VOLUM_CELL_EDGE_UL_KB,',
 'ROUND((23*SUM(NVL(L9102,0))+34*SUM(NVL(L9103,0))+40*SUM(NVL(L9104,0))+54*SUM(NVL(L9105,0)))/1024,2) TOT_VOLUM_CELL_GPRS_DL_KB,',
 'ROUND((22*SUM(NVL(L9302,0))+28*SUM(NVL(L9303,0))+37*SUM(NVL(L9304,0))+44*SUM(NVL(L9305,0))+56*SUM(NVL(L9306,0))+74*SUM(NVL(L9307,0))+56*SUM(NVL(L9308,0))+68*SUM(NVL(L9309,0))+74*SUM(NVL(L9310,0)))/1024,2) TOT_VOLUM_CELL_EDGE_DL_KB,',
 'ROUND(SUM(NVL(L9421,0))/1024,2) TOT_VOLUSERUM_CELL_GPRS_UL_KB,',
 'ROUND(SUM(NVL(L9423,0))/1024,2) TOT_VOLUSERUM_CELL_EDGE_UL_KB,',
 'ROUND(SUM(NVL(L9525,0))/1024,2) TOT_VOLUSERUM_CELL_GPRS_DL_KB,',
 'ROUND(SUM(NVL(L9527,0))/1024,2) TOT_VOLUSERUM_CELL_

## Parsing

We use <i><u>moz_sql_parser</u></i> for parsing SQL. Then it's parsed to json for more comfortable interaction.

In [5]:
parsed = json.dumps(moz_sql_parser.parse(q))
que = json.loads(parsed)
sample = que['select'][1]
sample

{'value': {'round': [{'div': [{'add': [{'mul': [22,
         {'sum': {'nvl': ['L9202', 0]}}]},
       {'mul': [28, {'sum': {'nvl': ['L9203', 0]}}]},
       {'mul': [37, {'sum': {'nvl': ['L9204', 0]}}]},
       {'mul': [44, {'sum': {'nvl': ['L9205', 0]}}]},
       {'mul': [56, {'sum': {'nvl': ['L9206', 0]}}]},
       {'mul': [74, {'sum': {'nvl': ['L9207', 0]}}]},
       {'mul': [56, {'sum': {'nvl': ['L9208', 0]}}]},
       {'mul': [68, {'sum': {'nvl': ['L9209', 0]}}]},
       {'mul': [74, {'sum': {'nvl': ['L9210', 0]}}]}]},
     1024]},
   2]},
 'name': 'TOT_VOLUM_CELL_EDGE_UL_KB'}

We create dictionary with every formula that passes given conditions. Counters with division by counter (if found) are ignored.
<br>
We use function <i><u>simplify</u></i> from custom library <i><u>tools</u></i> to create human readable formulas from json.
<br>
Then with <i><u>clean_formula</u></i> from custom library <i><u>tools</u></i> we get rid of all redundant parentheses.

In [6]:
t = {}

for i in que['select']:
    try:
        t[i.get('name',i.get('value'))] = clean_formula(''.join([i[0] for i in simplify(i.get('value', 'Null'), [])]))
    except Exception as e:
        print('Illigal operation in', i['name'])

Illigal operation in AU1
Illigal operation in AU2
Illigal operation in AD1
Illigal operation in AD2
Illigal operation in DATABASE_HRACT


## Results

In [7]:
for key in list(t.keys())[:10]:
    print(key + ':   ', t[key])

TOT_VOLUM_CELL_GPRS_UL_KB:    (23*L9002+34*L9003+40*L9004+54*L9005)/1024
TOT_VOLUM_CELL_EDGE_UL_KB:    (22*L9202+28*L9203+37*L9204+44*L9205+56*L9206+74*L9207+56*L9208+68*L9209+74*L9210)/1024
TOT_VOLUM_CELL_GPRS_DL_KB:    (23*L9102+34*L9103+40*L9104+54*L9105)/1024
TOT_VOLUM_CELL_EDGE_DL_KB:    (22*L9302+28*L9303+37*L9304+44*L9305+56*L9306+74*L9307+56*L9308+68*L9309+74*L9310)/1024
TOT_VOLUSERUM_CELL_GPRS_UL_KB:    L9421/1024
TOT_VOLUSERUM_CELL_EDGE_UL_KB:    L9423/1024
TOT_VOLUSERUM_CELL_GPRS_DL_KB:    L9525/1024
TOT_VOLUSERUM_CELL_EDGE_DL_KB:    L9527/1024
USERDATA_THR_TBF_AV_UL_KB_SEC_N1:    8*(L9421/1024+L9423/1024)
USERDATA_THR_TBF_AV_UL_KB_SEC_D1:    0.001*(A9014+A9214)


Creating a DataFrame for comfortable import into DB.

In [8]:
formulas_df = pd.DataFrame({'NUM_DEN':list(t.keys()), 'VENDOR':['H' for i in t.keys()], 
              'FORMULA':list(t.values()), 'DATE_START': "to_date('2018-01-01', 'YYYY-MM-DD')",
             'AG': ['SUM' for i in t.keys()], 'STANDART':['2G' for i in t.keys()], 'TYPE_STAT':['RAN' for i in t.keys()]})

In [9]:
formulas_df.head()

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART,TYPE_STAT
0,TOT_VOLUM_CELL_GPRS_UL_KB,H,(23*L9002+34*L9003+40*L9004+54*L9005)/1024,"to_date('2018-01-01', 'YYYY-MM-DD')",SUM,2G,RAN
1,TOT_VOLUM_CELL_EDGE_UL_KB,H,(22*L9202+28*L9203+37*L9204+44*L9205+56*L9206+...,"to_date('2018-01-01', 'YYYY-MM-DD')",SUM,2G,RAN
2,TOT_VOLUM_CELL_GPRS_DL_KB,H,(23*L9102+34*L9103+40*L9104+54*L9105)/1024,"to_date('2018-01-01', 'YYYY-MM-DD')",SUM,2G,RAN
3,TOT_VOLUM_CELL_EDGE_DL_KB,H,(22*L9302+28*L9303+37*L9304+44*L9305+56*L9306+...,"to_date('2018-01-01', 'YYYY-MM-DD')",SUM,2G,RAN
4,TOT_VOLUSERUM_CELL_GPRS_UL_KB,H,L9421/1024,"to_date('2018-01-01', 'YYYY-MM-DD')",SUM,2G,RAN


Total count of all counters that fits the condition.

In [10]:
len(list(formulas_df['NUM_DEN']))

109

### Import into DB

In [11]:
con, cur = connect('vertica')

In [12]:
for i in formulas_df.head(3).iterrows():
    cur.execute('''INSERT INTO VERTICA_STAT.NUM_DENOM (NUM_DEN, VENDOR, FORMULA, DATE_START, AG, STANDART, TYPE_STAT) VALUES ('{}', '{}', '{}', {}, '{}', '{}', '{}')'''.format(*i[1]))

In [13]:
pd.read_sql('select * from VERTICA_STAT.NUM_DENOM where standart = \'2G\'', con).head(3)

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART,TYPE_STAT
0,USERDATA_THR_TBF_AV_UL_KB_SEC_NUM,H,8*(L9421/1024+L9423/1024),2018-01-01,SUM,2G,RAN
1,USERDATA_THR_TBF_AV_UL_KB_SEC_DEN,H,0.001*(A9014+A9214),2018-01-01,SUM,2G,RAN
2,USERDATA_THR_TBF_AV_DL_KB_SEC_NUM,H,8*(L9525+L9527),2018-01-01,SUM,2G,RAN
